<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0
    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-19 14:42:30
-------------------
qualified stocks: 96
with latest results: 33
still star stocks: 19
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  55.89 K
Current:  1.28 C
-------------------
Today PnL: 52.62 K (0.41%)
Current PnL: -15.39 L (-10.91%)
CY Booked + Current PnL: -12.23 L (-8.67%)
-------------------
Total profit:  4.39 L
Total loss:  -19.77 L
-------------------
Total Booked + Current PnL: 14.75 L (12.94%)
Total Booked PnL: 30.14 L (26.43%)
Curr Year Booked PnL: 3.16 L (2.46%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 78.49 L (61.1%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.40 C (108.97%)
Deployed:  1.14 C
Current:  1.28 C
CAGR/XIRR %: 8.56%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SBILIFE,1496.49,1924.99,0.20,H-LC,70.83,148798.0,23093.0,12901.0,-0.11,18.37,8.67,28.63,159.0,1.79,1.16,32.56,AR,ATH,INSURANCE
63,RELIANCE,1288.53,1526.00,5.74,X-LC,82.29,227883.0,24295.0,13217.0,-0.97,11.93,5.80,18.43,161.0,1.84,1.78,24.13,XY25,NTT,REFINERIES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.58,182214.0,30850.0,14923.0,0.10,20.38,8.19,30.25,149.0,2.07,1.42,44.22,X40N,NTT,FINANCE
57,PIDILITIND,2504.06,3576.00,-16.90,X-LC,35.42,92124.0,17002.0,15154.0,-0.66,22.63,16.45,42.81,121.0,1.12,0.72,15.53,X40,BTT,CHEMICALS
71,SBIN,760.30,863.00,-14.39,M-LC,21.88,210715.0,9236.0,17974.0,0.39,4.58,8.53,13.51,60.0,0.51,1.65,17.76,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,STARHEALTH,551.56,761.0,0.02,M-SC,87.50,163677.0,-36539.0,112561.0,7.33,-18.25,68.77,37.97,108.0,-0.32,1.28,31.42,XY24,NTT,INSURANCE
90,VALIANTORG,512.64,838.0,-340.80,H-SC,64.58,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,145.0,-0.27,0.80,71.45,XR,NTT,CHEMICALS
21,COFFEEDAY,59.14,80.0,-59.76,L-SC,76.04,58656.0,-54893.0,94946.0,4.98,-48.34,161.87,35.27,181.0,-0.58,0.46,43.56,XR,NTT,HOTELS
84,TITAGARH,1097.23,1548.0,-4.75,H-SC,98.96,167248.0,-24767.0,103660.0,4.69,-12.90,61.98,41.08,119.0,-0.24,1.31,41.55,XY24,NTT,ENGINEERING
12,BANKINDIA,113.49,190.0,-31.87,M-MC,16.67,134284.0,5132.0,81940.0,3.21,3.97,61.02,67.42,57.0,0.06,1.05,30.60,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,558.54,680.00,-16.80,H-MC,37.50,178566.0,1509.0,36999.0,-2.22,0.85,20.72,21.75,183.0,0.04,1.40,27.73,XY24,NTT,PAINTS
30,GREENPANEL,375.16,537.00,171.32,M-SC,47.92,121024.0,-62054.0,141029.0,-2.12,-33.89,116.53,43.14,109.0,-0.44,0.95,13.84,XY24,NTT,MISC
36,HONAUT,42646.15,58518.52,-21.89,X-MC,79.17,114390.0,-13548.0,61164.0,-1.78,-10.59,53.47,37.22,110.0,-0.22,0.89,16.78,X40N,ATH,ELECTRICAL
54,MEDANTA,1022.60,1486.00,1.87,X-MC,3.12,179558.0,24123.0,46308.0,-1.65,15.52,25.79,45.32,201.0,0.52,1.40,22.17,XY24,NTT,HEALTHCARE
42,INFY,1432.81,2275.00,-12.78,X-LC,66.67,217424.0,18263.0,98797.0,-1.62,9.17,45.44,58.78,6.0,0.18,1.70,13.52,X40,BTT,IT


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIAMART,2311.97,4911.36,-51.47,H-SC,59.38,104874.0,3147.0,111229.0,1.42,3.09,106.06,112.43,1.0,0.03,0.82,23.78,AR,ATH,MISC
83,TCS,3928.29,4998.00,-15.84,X-LC,60.42,189988.0,-22140.0,79909.0,-1.21,-10.44,42.06,27.23,3.0,-0.28,1.49,8.88,X40,BTT,IT
82,TATAMOTORS,776.02,1065.00,-51.41,M-LC,92.71,193728.0,-11917.0,88495.0,0.05,-5.80,45.68,37.24,4.0,-0.13,1.51,26.10,XY24,NTT,AUTO
10,AWL,342.88,485.00,-68.50,H-MC,4.17,170942.0,-46787.0,137027.0,-0.43,-21.49,80.16,41.45,4.0,-0.34,1.34,14.19,XY24,NTT,FMCG
42,INFY,1432.81,2275.00,-12.78,X-LC,66.67,217424.0,18263.0,98797.0,-1.62,9.17,45.44,58.78,6.0,0.18,1.70,13.52,X40,BTT,IT


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,DMART,3484.00,5112.00,-17.84,X-LC,14.58,101165.0,17549.0,21528.0,0.61,20.99,21.28,46.73,162.0,0.82,0.79,23.83,AR,NTT,FMCG
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.58,182214.0,30850.0,14923.0,0.10,20.38,8.19,30.25,149.0,2.07,1.42,44.22,X40N,NTT,FINANCE
57,PIDILITIND,2504.06,3576.00,-16.90,X-LC,35.42,92124.0,17002.0,15154.0,-0.66,22.63,16.45,42.81,121.0,1.12,0.72,15.53,X40,BTT,CHEMICALS
58,POLYCAB,5005.92,7541.35,9.62,H-MC,88.54,171304.0,31138.0,39845.0,0.39,22.22,23.26,50.65,56.0,0.78,1.34,30.74,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-1.13,M-LC,84.38,191129.0,39923.0,25745.0,-0.87,26.40,13.47,43.43,48.0,1.55,1.49,32.48,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,LUXIND,1448.19,2442.00,45.26,M-SC,73.96,150822.0,13244.0,81172.0,1.49,9.63,53.82,68.62,66.0,0.16,1.18,30.75,X40N,NTT,TEXTILES
1,AARTIIND,487.04,919.00,41.80,M-SC,91.67,81421.0,85.0,72049.0,2.72,0.10,88.49,88.69,169.0,0.00,0.64,37.30,XR,NTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.58,182214.0,30850.0,14923.0,0.10,20.38,8.19,30.25,149.0,2.07,1.42,44.22,X40N,NTT,FINANCE
58,POLYCAB,5005.92,7541.35,9.62,H-MC,88.54,171304.0,31138.0,39845.0,0.39,22.22,23.26,50.65,56.0,0.78,1.34,30.74,X40N,ATH,CABLES
29,GILLETTE,8109.44,10330.69,-0.40,H-SC,77.08,146786.0,8926.0,28829.0,0.28,6.47,19.64,27.39,64.0,0.31,1.15,28.49,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-1.33,H-SC,20.31,110972.0,8968.0,36055.0,1.86,8.79,32.49,44.14,31.0,0.25,0.87,26.46,XR,ATH,MISC
12,BANKINDIA,113.49,190.00,-31.87,M-MC,16.67,134284.0,5132.0,81940.0,3.21,3.97,61.02,67.42,57.0,0.06,1.05,30.60,XR,NTT,BANKS
95,WIPRO,243.46,420.00,-8.80,M-LC,58.33,156116.0,5171.0,104285.0,-0.99,3.43,66.80,72.51,49.0,0.05,1.22,17.54,XR,NTT,IT
39,INDIAMART,2311.97,4911.36,-51.47,H-SC,59.38,104874.0,3147.0,111229.0,1.42,3.09,106.06,112.43,1.0,0.03,0.82,23.78,AR,ATH,MISC
34,HEROMOTOCO,4311.81,6039.03,-0.48,H-MC,85.42,152086.0,1173.0,59283.0,0.47,0.78,38.98,40.06,27.0,0.02,1.19,24.40,AR,ATH,AUTO


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-340.80,H-SC,64.58,102284.0,-31002.0,115591.0,4.99,-23.26,113.01,63.47,145.0,-0.27,0.80,71.45,XR,NTT,CHEMICALS
87,UJJIVANSFB,52.77,60.00,40.02,M-SC,33.33,121149.0,-21330.0,40851.0,0.63,-14.97,33.72,13.70,213.0,-0.52,0.95,42.99,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-31.87,M-MC,16.67,134284.0,5132.0,81940.0,3.21,3.97,61.02,67.42,57.0,0.06,1.05,30.60,XR,NTT,BANKS
70,SBILIFE,1496.49,1924.99,0.20,H-LC,70.83,148798.0,23093.0,12901.0,-0.11,18.37,8.67,28.63,159.0,1.79,1.16,32.56,AR,ATH,INSURANCE
88,UNIONBANK,123.87,163.00,-11.04,M-LC,63.54,155894.0,15054.0,29433.0,3.07,10.69,18.88,31.59,70.0,0.51,1.22,35.42,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,718.00,-869.30,H-SC,100.00,73320.0,-21996.0,66692.0,-1.38,-23.08,90.96,46.89,173.0,-0.33,0.57,44.62,X40N,NTT,MISC
84,TITAGARH,1097.23,1548.00,-4.75,H-SC,98.96,167248.0,-24767.0,103660.0,4.69,-12.90,61.98,41.08,119.0,-0.24,1.31,41.55,XY24,NTT,ENGINEERING
80,TANLA,917.30,1963.11,-37.80,H-SC,97.92,152346.0,-87069.0,360024.0,2.13,-36.37,236.32,114.01,20.0,-0.24,1.19,39.72,AR,ATH,IT
76,SONACOMS,578.05,1006.00,-27.23,M-MC,96.88,94027.0,-7132.0,82020.0,-1.02,-7.05,87.23,74.03,43.0,-0.09,0.74,32.14,AR,BTT,AUTO
94,WHIRLPOOL,1151.42,2270.00,-35.17,M-SC,95.83,163233.0,17003.0,125053.0,-0.86,11.63,76.61,97.15,28.0,0.14,1.28,39.45,X40,NTT,DURABLES


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,16.98
1,25,38.32
2,50,67.54


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.56
X40N    15.23
X40     14.16
AR      13.96
XR      10.41
XY25     9.37
X200     1.18
SR       1.10
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.62
M-SC    17.70
X-LC    15.95
H-MC     8.91
M-LC     8.25
H-LC     5.83
M-MC     5.52
X-MC     5.51
X-SC     2.80
L-SC     2.27
L-LC     1.16
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.58,-0.14,32.00
BANKS,10.91,-8.05,61.04
IT,9.05,-19.52,86.20
FINANCE,7.48,-15.65,67.84
MISC,7.13,-24.21,77.66
INSURANCE,6.13,1.70,27.93
PAINTS,5.49,-16.96,51.57
AUTO,4.33,-9.25,56.87
ELECTRICAL,4.08,-6.52,60.80


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2930804.0,27
AR,1277370.0,15
XR,1214019.0,15
X40N,1211444.0,18
X40,783432.0,12
SR,203951.0,2
XY25,160907.0,6
X200,66728.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2294472.0,22
M-SC,1974036.0,21
X-LC,701739.0,12
M-MC,562129.0,5
H-MC,468651.0,8
X-SC,349356.0,4
M-LC,324565.0,6
L-SC,307275.0,4
X-MC,298897.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      880884.0      6
M-SC       XY24      745695.0      6
H-SC       AR        530561.0      3
           X40N      519724.0      7
M-SC       X40N      438404.0      6
           XR        418005.0      5
M-MC       XY24      398169.0      3
X-LC       X40       380852.0      5
H-SC       XR        276718.0      4
M-LC       XY24      202306.0      4
H-LC       AR        197270.0      4
M-SC       X40       195482.0      2
X-LC       AR        180387.0      3
H-MC       XY24      174026.0      2
L-SC       XR        159508.0      2
X-MC       XY24      144408.0      2
H-MC       AR        131404.0      2
X-SC       SR        119674.0      1
           XR        115746.0      1
           X40N      113936.0      2
M-LC       XR        104285.0      1
X-MC       X40N       99535.0      2
M-SC       AR         92173.0      1
H-SC       X40        86585.0      2
M-SC       SR         84277.0      1
L-SC       XY24       84212.0      1
M-MC       AR         82020.0      1
           XR         81940.0      1
X-LC       XY25       73772.0      3
           X200       66728.0      1
H-MC       X40        65559.0      2
L-SC       AR         63555.0      1
H-MC       XR         57817.0      1
X-MC       X40        54954.0      1
L-LC       XY25       40824.0      1
H-MC       X40N       39845.0      1
H-LC       XY25       28337.0      1
M-LC       XY25       17974.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
